# Octopy Features Generation

Vamos a crear las features que me permite obtener Octopy estas son las siguientes:

* Coefs para equipo local y visitante
* Probs para equipo local y visitante

In [1]:
import warnings
warnings.filterwarnings('ignore')

import sqlite3
import pandas as pd
from datetime import *
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from logistic_model import LogisticRegression, LogisticModel
from implied import ImpliedProbability
from sklearn.metrics import plot_confusion_matrix, accuracy_score

In [2]:
def connect_database(db):
    con = sqlite3.connect(db)
    cursor = con.cursor()
    return cursor, con


def leer_partidos(con):
    partidos = pd.read_sql_query('SELECT * FROM partidos', con) #, index_col='fecha')
    return partidos

cur, con = connect_database('../../Clasificacion.db')

partidos = leer_partidos(con)

In [3]:
partidos['fecha'] = partidos['timestamp']
partidos['fecha'] = partidos['fecha'].map(lambda X: int(datetime.fromtimestamp(int(X)).date().strftime('%Y%m%d')))
partidos = partidos.sort_values(by='fecha')

In [4]:
temporadas = partidos['temporada'].unique()
dias = partidos['fecha'].unique()
divisiones = partidos['division'].unique()

In [5]:
for div in divisiones:
    partidos_div = partidos[partidos['division'] == div]

    for temp in temporadas:
        partidos_temp = partidos_div[partidos_div['temporada'] == temp]
        equipos = partidos_temp['equipo_local'].unique()
        dias = partidos_temp[partidos_temp['jornada'] > 2]['fecha'].unique()

        for d in dias:
            partidos_dia = partidos_temp[partidos_temp['fecha'] <= d]
            model = LogisticModel()
            model.fit(partidos_dia['equipo_local'],partidos_dia['equipo_visitante'],partidos_dia['goles_local'],partidos_dia['goles_visitante'])
            coefs = model.get_coef()
            partidos_hoy = partidos_dia[partidos_dia['fecha'] == d]
            for index, row in partidos_hoy.iterrows():
                partidos.at[partidos['id_partido'] == row.id_partido, 'local_home_wins'] = \
                    coefs.loc['home_'+row.equipo_local]['home wins'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'local_draw'] = \
                    coefs.loc['home_'+row.equipo_local]['draw'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'local_away_wins'] = \
                    coefs.loc['home_'+row.equipo_local]['away wins'][0]

                partidos.at[partidos['id_partido'] == row.id_partido, 'visitante_home_wins'] = \
                    coefs.loc['away_'+row.equipo_local]['home wins'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'visitante_draw'] = \
                    coefs.loc['away_'+row.equipo_local]['draw'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'visitante_away_wins'] = \
                    coefs.loc['away_'+row.equipo_local]['away wins'][0]

                partidos.at[partidos['id_partido'] == row.id_partido, 'probs_equipo_local'] = \
                    model.predict_proba(row.equipo_local, row.equipo_visitante)[row.equipo_local][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'probs_equipo_visitante'] = \
                    model.predict_proba(row.equipo_local, row.equipo_visitante)[row.equipo_visitante][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'probs_draw'] = \
                    model.predict_proba(row.equipo_local, row.equipo_visitante)['draw'][0]

        print(temp + ' ' + str(div))


1970-71 2
1971-72 2
1972-73 2


In [6]:
partidos.to_csv('partidos_with_octopy_probabilites.csv')

Convert String Date to Timestamp

In [15]:
int(datetime.strptime('21/04/2021', '%d/%m/%Y').strftime("%s"))

1618956000